In [ ]:
import os
import torch
import numpy as np

# CODE FILES HERE
from model_params import get_model_data_tdcvae
from models.tdcvae.tdcvae import TD_Cvae
from solver import Solver
from dataloader import DataLoader
from directories import Directories
from plots import plot_losses, plot_gaussian_distributions, plot_rl_kl, plot_latent_space,\
plot_latent_manifold, plot_prepro_alpha_params_distribution, plot_prepro_radius_params_distribution,\
plot_transformed_images, plot_y_space_thetas, plot_y_space_scales
from auxiliary import get_latent_spaces, produce_ys, produce_ys_zs
from preprocessing import RandomPreprocessing, preprocess_batch_det

# SETTINGS HERE
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # to see the CUDA stack
%matplotlib inline
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
# supress cluttering warnings in solutions
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

In [ ]:
# only mnist available
dataset_arg = "mnist"
data = get_model_data_tdcvae(dataset_arg)

In [ ]:
directories = Directories("tdcvae", dataset_arg.lower(), data["z_dim"], make_dirs=False)
data_loader = DataLoader(directories, data["batch_size"], dataset_arg.lower(), thetas=data["thetas"], scales=data["scales"])
model = TD_Cvae(data_loader.input_dim, data_loader.input_dim, data["z_dim"], data["beta"], rotations=True, scaling=True)
solver = Solver(model, data_loader, data["optimizer"], data["epochs"], data["optim_config"], step_config=data["step_config"],\
                lr_scheduler=data["lr_scheduler"], tdcvae_mode=True, save_model_state=False)
solver.main()

In [ ]:
# Uncomment to load a model to continue training.
#solver = torch.load("../results/tdcvae/MNIST_z=2_0/model_state.pt")
#solver.main()

In [ ]:
# Uncomment to load a trained model for inference.
solver = torch.load("../../results/results_slurm/test_thing2/0/model_state.pt", map_location='cpu')
#solver = torch.load("../../results/TEST2/model_state.pt", map_location='cpu')
solver.model.eval()
solver.data_loader.dataset = solver.data_loader.dataset.lower()
#solver.data_loader.directories.make_dirs = "../../results/TEST2/"
#solver.data_loader.directories.result_dir = "../../results/TEST2/"
#solver.data_loader.directories.make_dirs = "../../results_slurm/test_thing2/mnist_z=2_0/"
#solver.data_loader.directories.result_dir = "../../results_slurm/test_thing2/"
solver.data_loader.directories.make_dirs = "../../results/results_slurm/test_thing2/0/"
solver.data_loader.directories.result_dir = "../../results/results_slurm/test_thing2/0/"
print(solver.data_loader.thetas, solver.data_loader.scales)
rotations = solver.data_loader.thetas is not None
scaling = solver.data_loader.scales is not None
print(rotations, scaling)
model = TD_Cvae(784, 784, 2, solver.model.beta, rotations=rotations, scaling=scaling)
model.encoder.load_state_dict(solver.model.encoder.state_dict())
model.decoder.load_state_dict(solver.model.decoder.state_dict())

In [ ]:
# Plotting train and test losses for all epochs
plot_losses(solver, solver.train_loss_history["train_loss_acc"], solver.test_loss_history)

In [ ]:
# Plotting the gaussian of z space and some metrics about the space
plot_gaussian_distributions(solver, len(solver.train_loss_history["train_loss_acc"]))

In [ ]:
# Monitoring the reconstruction loss (likelihood lower bound) and KL divergence
DEBUG = 1
if DEBUG:
    for epoch, train_loss, test_loss, rl, kl in zip(solver.train_loss_history["epochs"], \
        solver.train_loss_history["train_loss_acc"], solver.test_loss_history, \
        solver.train_loss_history["recon_loss_acc"], solver.train_loss_history["kl_diverg_acc"]):
        print("epoch: {}, train_loss: {:.2f}, test_loss: {:.2f}, recon. loss: {:.2f}, KL div.: {:.2f}".format(
            epoch, train_loss, test_loss, rl, kl))
plot_rl_kl(solver, solver.train_loss_history["recon_loss_acc"], solver.train_loss_history["kl_diverg_acc"])

In [ ]:
# defaults settings that are hardcoded in the code in case someone does run tdcvae but not provides theta/scale parameters
if solver.data_loader.thetas:
    if solver.data_loader.thetas["theta_1"] is None or solver.data_loader.thetas["theta_2"] is None:
        theta_range_1 = [-np.pi, np.pi]
        theta_range_2 = [-np.pi/4, np.pi/4]
    else:
        theta_range_1 = solver.data_loader.thetas["theta_1"]
        theta_range_2 = solver.data_loader.thetas["theta_2"]
if solver.data_loader.scales:
    if solver.data_loader.scales["scale_1"] is None or solver.data_loader.scales["scale_2"] is None:
        scale_range_1 = [0.85, 1.15]
        scale_range_2 = [-0.15, 0.15]
    else:
        scale_range_1 = solver.data_loader.scales["scale_1"]
        scale_range_2 = solver.data_loader.scales["scale_2"]

In [ ]:
# make random transformations
num_test_samples = solver.data_loader.num_test_samples # put like 1000 if sample with single data point / specific class
if solver.data_loader.thetas and solver.data_loader.scales:
    rand_transformation = RandomPreprocessing(num_test_samples, solver.data_loader.img_dims,\
                                    theta_range_1, theta_range_2,\
                                    scale_range_1, scale_range_2)
elif solver.data_loader.thetas:
    rand_transformation = RandomPreprocessing(num_test_samples, solver.data_loader.img_dims,\
                                    theta_range_1=theta_range_1, theta_range_2=theta_range_2)
elif solver.data_loader.scales:
    rand_transformation = RandomPreprocessing(num_test_samples, solver.data_loader.img_dims,\
                                    scale_range_1=scale_range_1, scale_range_2=scale_range_2)
mode = "tdcvae"
single_point_sampler = ("single_point", num_test_samples)
specific_class_sampler = ("specific_class", 1, num_test_samples)
test_loader = solver.data_loader.get_new_test_data_loader() # insert one of the samplers above if want to use it
z_space, y_space, _ = get_latent_spaces(model, mode, test_loader, num_test_samples,\
                                        solver.model.z_dim, int(num_test_samples/len(test_loader)),\
                                        solver.data_loader.with_labels,\
                                        solver.data_loader.input_dim, rand_transformation)

In [ ]:
if solver.data_loader.thetas:
    theta_ticks_y = np.arange(np.degrees(theta_range_1[0]), np.degrees(theta_range_1[1])+1, 30).tolist()
    theta_ticks_z = np.arange(np.degrees(theta_range_2[0]), np.degrees(theta_range_2[1])+1, 10).tolist()
if solver.data_loader.scales:
    scale_ticks_y = np.linspace(scale_range_1[0], scale_range_1[1], 13).tolist()
    scale_ticks_z = np.linspace(scale_range_2[0], scale_range_2[1], 4).tolist()

In [ ]:
# visualize q(z) (latent space z)
if solver.model.z_dim == 2 and solver.data_loader.thetas or solver.data_loader.scales:
    bs = solver.data_loader.batch_size
    print("\mu(z_1)={}, var(z_1)={}".format(np.mean(z_space[:, 0]), np.var(z_space[:, 0])))
    print("\mu(z_2)={}, var(z_2)={}".format(np.mean(z_space[:, 1]), np.var(z_space[:, 1])))
    print("\mu(z_1|x)={}, var(z_1|x)={}".format(np.mean(z_space[:bs, 0]), np.var(z_space[:bs, 0])))
    print("\mu(z_2|x)={}, var(z_2|x)={}".format(np.mean(z_space[:bs, 1]), np.var(z_space[:bs, 1])))
    if solver.data_loader.thetas:
        labels = np.degrees(rand_transformation.prepro_params["theta_diff"]).tolist()
        plot_latent_space(solver, z_space, ticks=theta_ticks_z, var="z", title="theta", labels=labels,\
                          colors=len(theta_ticks_z)-1)
    if solver.data_loader.scales:
        labels = rand_transformation.prepro_params["scale_diff"].tolist()
        plot_latent_space(solver, z_space, ticks=scale_ticks_z, var="z", title="scale", labels=labels,\
                          colors=len(scale_ticks_z)-1, transpose=False)
else:
    print("Plot of latent space not possible as dimension of z is not 2")

In [ ]:
# Visualize q(y)
if solver.model.z_dim == 2 and solver.data_loader.thetas or solver.data_loader.scales:
    bs = solver.data_loader.batch_size
    print("\mu(y_1)={}, var(y_1)={}".format(np.mean(y_space[0:bs, 0]), np.var(y_space[0:bs, 0])))
    print("\mu(y_2)={}, var(y_2)={}".format(np.mean(y_space[0:bs, 1]), np.var(y_space[0:bs, 1])))
    print("\mu(y_1|x)={}, var(y_1|x)={}".format(np.mean(y_space[:, 0]), np.var(y_space[:, 0])))
    print("\mu(y_2|x)={}, var(y_2|x)={}".format(np.mean(y_space[:, 1]), np.var(y_space[:, 1])))
    if solver.data_loader.thetas:
        labels = np.degrees(rand_transformation.prepro_params["theta_1"]).tolist()
        plot_latent_space(solver, y_space, ticks=theta_ticks_y, var="y", title="theta", labels=labels)
    if solver.data_loader.scales:
        labels = rand_transformation.prepro_params["scale_1"].tolist()
        plot_latent_space(solver, y_space, ticks=scale_ticks_y, var="y", title="scale", labels=labels, transpose=False)
else:
    print("Plot of latent space not possible as dimension of z is not 2")

In [ ]:
# Visualizations of learned q(z) for generative models with two-dimensional latent space
if solver.model.z_dim == 2:
    n = 11
    if solver.data_loader.thetas and solver.data_loader.scales:
        grid_x = np.linspace(-2, 2, n)
        grid_y = np.linspace(-2, 2, n)
    elif solver.data_loader.thetas:
        grid_x = np.linspace(-2, 2, n)
        grid_y = np.linspace(-2, 2, n)
    elif solver.data_loader.scales:
        grid_x = np.linspace(-2, 2, n)
        grid_y = np.linspace(-1, -10, n)
    test_loader = solver.data_loader.get_new_test_data_loader()
    x_t, class_labels = iter(test_loader).next()
    x_t, _ = rand_transformation.preprocess_samples(x_t, save=False)
    for i in range(3):
        plot_latent_manifold(model.decoder, solver, "bone", grid_x, grid_y, n=n, x_t=x_t[i].view(1, 784),\
                             label=class_labels[i], center_rect=(solver.data_loader.scales is None))
else:
    print("Plot is not possible as dimension of z is not 2 or model is loaded")

In [ ]:
# Experiments to check the z space for e.g. a single sample and then see from what the sample are drawn.
if solver.model.z_dim == 2:
    num_samples = 1
    #single_point_sampler = ("single_point", num_samples)
    specific_class_sampler = ("specific_class", 7, num_samples)
    test_loader = solver.data_loader.get_new_test_data_loader(specific_class_sampler)
    x, labels = iter(test_loader).next()
    x_t = torch.zeros((num_samples, 1, 28, 28))
    for idx, x in enumerate(test_loader):
        x, label = x[0], x[1]
        x_t[idx] = x
    x_next = x_t[:num_samples].clone().detach()
    scales_1 = np.linspace(1.0, 1.0, 1)
    scales_2 = np.linspace(1.0, 1.0, 1)
    thetas_1 = np.linspace(np.radians(0), np.radians(0), 100)
    thetas_2 = np.linspace(-np.pi/4, np.pi/4, 100)
    ys1, zs1 = produce_ys_zs(model, x_t.shape[0], x_t.view(-1, 784).numpy(), scales_1, thetas_1, x_next.view(-1, 784).numpy(), scales_2, thetas_2)
    ys1 = np.reshape(ys1, (np.prod(ys1.shape[:-1]), 2))
    zs1 = np.reshape(zs1, (np.prod(zs1.shape[:-1]), 2))
    x_t = preprocess_batch_det(x_t[0:1], thetas_1, scales_1)
    x_t = torch.FloatTensor(x_t).view(1, solver.data_loader.input_dim)
    n = 11
    grid_x = np.linspace(-2, 2, n)
    grid_y = np.linspace(-2, 2, n)
    #grid_x = np.linspace(np.min(zs1[:, 0]), np.max(zs1[:, 0]), n)
    #grid_y = np.linspace(np.min(zs1[:, 1]), np.max(zs1[:, 1]), n)
    #plot_latent_space(solver, ys1, ticks=theta_ticks_y, var="y", title="theta",\
    #                  labels=np.repeat(np.degrees(thetas_1), num_samples), xticks=grid_x, yticks=grid_y)
    plot_latent_space(solver, zs1, ticks=theta_ticks_z, var="z", title="theta",\
                      labels=np.repeat(np.degrees(thetas_2), num_samples), xticks=grid_x, yticks=grid_y,\
                      colors=len(theta_ticks_z)-1)
    #plot_latent_space(solver, zs1, ticks=theta_ticks_z, var="z", title="theta",\
    #                  labels=np.repeat(np.degrees(scales_2), num_samples), xticks=grid_x, yticks=grid_y,\
    #                  colors=len(scale_ticks_z)-1)
    plot_latent_manifold(model.decoder, solver, "bone", grid_x, grid_y, n=n, x_t=x_t, label=label,\
                         zdata=(zs1, np.repeat(np.degrees(thetas_2), num_samples), len(theta_ticks_z)-1))

In [ ]:
# show how the transformation actually produces results in practice.
test_loader = solver.data_loader.get_new_test_data_loader()
file_name = solver.data_loader.directories.result_dir + "/plot_transformed_images_"\
            + solver.data_loader.dataset + ".png"
plot_transformed_images(test_loader, solver.data_loader.batch_size, save_plot=solver.data_loader.directories.make_dirs, file_name=file_name)

In [ ]:
# Produce y space with deterministic transformation settings.
if solver.data_loader.thetas and solver.data_loader.scales and solver.model.z_dim == 2:
    test_loader = solver.data_loader.get_new_test_data_loader()
    num_rotations = 30
    num_scales = 30
    num_samples = 20
    test_loader = solver.data_loader.get_new_test_data_loader()
    x_t, targets = iter(test_loader).next()
    x_t = x_t.view(-1, 784).numpy()
    thetas = np.linspace(0, 1, num_rotations) * 2 * np.pi
    scales = 0.7 + np.linspace(0, 1, num_scales) * 0.6
    ys = produce_ys(model.encoder, x_t, scales, thetas, num_samples)
    print("Labels of data", targets[:num_samples])

In [ ]:
#ys = torch.load("../../results_slurm/test_thing2/ys.pt")
#thetas = np.linspace(0, 1, 30) * 2 * np.pi
#scales = 0.7 + np.linspace(0, 1, 30) * 0.6

In [ ]:
# scatter plot of the ys with thetas
if solver.data_loader.thetas and solver.data_loader.scales and solver.model.z_dim == 2 and ys is not None:
    ticks = np.arange(np.degrees(thetas[0]), np.degrees(thetas[-1])+1, 30).tolist()
    labels = np.degrees(thetas)
    print("Thetas applied: {}".format(labels))
    file_name = solver.data_loader.directories.result_dir + "/plot_y_space_thetas_fixed_"\
                + solver.data_loader.dataset + "_z=" + str(solver.model.z_dim) + ".png"
    plot_y_space_thetas(ys, ticks, labels, solver.data_loader.directories.make_dirs, file_name, solver.data_loader.dataset)

In [ ]:
# scatter plot of the ys with scales
if solver.data_loader.thetas and solver.data_loader.scales and solver.model.z_dim == 2 and ys is not None:
    labels = scales
    ticks = np.linspace(scales[0], scales[-1], 13).tolist()
    ticks = np.around(ticks, decimals=2)
    print("Scales: {}".format(labels))
    file_name = solver.data_loader.directories.result_dir + "/plot_y_space_scales_fixed_"\
                + solver.data_loader.dataset + "_z=" + str(solver.model.z_dim) + ".png"
    plot_y_space_scales(ys, ticks, labels, solver.data_loader.directories.make_dirs, file_name, solver.data_loader.dataset)

In [ ]:
# for rotation
if solver.model.z_dim == 2 and solver.data_loader.thetas and solver.data_loader.scales and ys is not None:
    file_name = solver.data_loader.directories.result_dir + "/plot_prepro_alpha_params_distribution_" \
                + solver.data_loader.dataset + "_z=" + str(solver.model.z_dim) + ".png"
    plot_prepro_alpha_params_distribution(ys, thetas, \
                                          solver.data_loader.directories.make_dirs, file_name,\
                                          solver.data_loader.dataset)

In [ ]:
# for scaling
if solver.model.z_dim == 2 and solver.data_loader.thetas and solver.data_loader.scales and ys is not None:
    file_name = solver.data_loader.directories.result_dir + "/plot_prepro_radius_params_distribution_" \
                + solver.data_loader.dataset + "_z=" + str(solver.model.z_dim) + ".png"
    plot_prepro_radius_params_distribution(ys, scales,\
                                          solver.data_loader.directories.make_dirs, file_name,\
                                          solver.data_loader.dataset)